In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import xmltodict
import pickle
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

#from toy_dataset import contexts

max_len = 512
num_layers = 24
d_model = 4096

In [48]:
tokenizer2 = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=False, add_special_tokens=False)

In [55]:
tokenizer.encode("John travelled")

[1079, 3, 21043, 1]

In [56]:
tokenizer.tokenize("John travels")

['▁John', '▁travel', 's']

In [51]:
print(tokenizer2.decode([3, 21043, 1]))
print(tokenizer2.decode([21043, 1]))

travelled</s>
travelled</s>


In [33]:
tokenizer.vocab

{'▁Weekend': 20418,
 '▁Children': 4351,
 '▁hiring': 7539,
 'Schraub': 31976,
 'liés': 22263,
 '▁submitted': 5776,
 'écrire': 21640,
 'RES': 12200,
 'kamagra': 31603,
 '▁convinced': 12718,
 'IMA': 26330,
 '▁Portrait': 25634,
 '▁Weihnachten': 29270,
 '▁inquiries': 20182,
 '▁Ru': 2770,
 '▁anymore': 7595,
 '▁practise': 25420,
 '▁fle': 8079,
 '▁appreciated': 9332,
 '▁reconciliation': 28326,
 'destinate': 30530,
 'Regulatory': 31056,
 '▁spreadsheet': 18527,
 'ezimal': 21649,
 '▁Restoration': 24560,
 '▁Muzic': 30555,
 'Biserica': 21287,
 '▁stole': 26357,
 '▁Kai': 17654,
 'size': 7991,
 'bourg': 21054,
 '▁spend': 1492,
 '▁trotz': 18732,
 'rind': 13119,
 '▁Johann': 19719,
 '▁Correct': 28223,
 '▁fighting': 6237,
 'Energi': 28452,
 '▁today': 469,
 '▁Karten': 20012,
 '▁volet': 15667,
 '▁cheveux': 18348,
 '▁Best': 1648,
 '▁Wur': 22977,
 '▁revolt': 28495,
 '▁Las': 7263,
 'extrêmement': 27666,
 'onul': 10197,
 'MC': 3698,
 '▁desfășur': 20827,
 '▁particulière': 23650,
 '▁constitute': 11708,
 '▁HTTP': 

In [43]:
tokenizer.vocab["travelled"]

21043

In [30]:
tokenizer.vocab[]

KeyError: ' '

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp")#, device_map="auto", load_in_8bit=True)

In [19]:
def find_index_one(input_ids, entity_str, index):
    
    entity_id = tokenizer.encode(entity_str)
    
    if len(entity_id) != 2:
        print("Not an appropriate entity!")
        return
    
    entity_id = entity_id[0]
    
    input_ids_list = input_ids.tolist()

    all_entity_mention_indices = []
    for i, j in enumerate(input_ids_list[0]):
        if j == entity_id:
            all_entity_mention_indices.append(i)
    try:
        entity_ind = all_entity_mention_indices[index]
        return entity_ind
    except:
        print("entity not found in the input!")
        return

In [56]:
def current_timestep_regular(context, answer_choices, question, template):
    
    prompt = template.render(context=context, question=question, answer_choices=answer_choices)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    out = model.generate(input_ids, max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
    next_token_scores = torch.nn.functional.softmax(
                    out.scores[0], dim=-1
                )  # (batch_size * num_beams, vocab_size)

    loc_tokens = []
    for location in answer_choices:
        loc_tokens.append(tokenizer.encode(location)[0])

    probabilities = {}
    for loc_token in loc_tokens:
        probabilities[tokenizer.decode(loc_token)] = next_token_scores[0][loc_token].item()

    the_sum = sum(probabilities.values())
    for key, value in probabilities.items():
        probabilities[key] = value/the_sum

    scores = []
    for i, score in enumerate(next_token_scores[0]):
        scores.append( (i, score.item()) )

    scores.sort(key=lambda x: x[1], reverse=True)
    scores = [(tokenizer.decode(a), b) for a, b in scores]

    return probabilities, scores

def previous_timestep(context, entity):
    
    input_ids = tokenizer.encode(context, return_tensors="pt")
    len_input_ids = len(input_ids[0])
    out = model.encoder(input_ids, output_special=True, output_hidden_states=True)
    special_hidden = out.special_hidden_states # 24 x (1, T, d)

    special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
    for i, hidden in enumerate(special_hidden):
        special_reformatted[i:i+1, :, :] = hidden
    
    entity_ind = find_index_one(input_ids, entity, 0)
    entity_hidden_state = special_reformatted[:, entity_ind, :].unsqueeze(0)
    
    return entity_hidden_state

def previous_timestep_with_template(context, answer_choices, question, template):
    
    prompt = template.render(context=context, question=question, answer_choices=answer_choices)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    len_input_ids = len(input_ids[0])
    out = model.encoder(input_ids, output_special=True, output_hidden_states=True)
    special_hidden = out.special_hidden_states # 24 x (1, T, d)

    special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
    for i, hidden in enumerate(special_hidden):
        special_reformatted[i:i+1, :, :] = hidden
    
    entity_ind = find_index_one(input_ids, "John", 0)
    entity_hidden_state = special_reformatted[:, entity_ind, :].unsqueeze(0)
    
    return entity_hidden_state

def current_timestep_enhanced(context, answer_choices, question, template, entity_hidden_state):
    prompt = template.render(context=context, question=question, answer_choices=answer_choices)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    entity_inds = [ find_index_one(input_ids, "John", 0) ]
    out = model.generate(
        input_ids=input_ids,
        entity_hidden_states=entity_hidden_state,
        entity_inds=entity_inds,
        max_new_tokens=1,
        return_dict_in_generate=True,
        output_scores=True
    ) 
    next_token_scores = torch.nn.functional.softmax(
                    out.scores[0], dim=-1
                )  # (batch_size * num_beams, vocab_size)

    loc_tokens = []
    for location in answer_choices:
        loc_tokens.append(tokenizer.encode(location)[0])

    probabilities = {}
    for loc_token in loc_tokens:
        probabilities[tokenizer.decode(loc_token)] = next_token_scores[0][loc_token].item()

    the_sum = sum(probabilities.values())
    for key, value in probabilities.items():
        probabilities[key] = value/the_sum

    scores = []
    for i, score in enumerate(next_token_scores[0]):
        scores.append( (i, score.item()) )

    scores.sort(key=lambda x: x[1], reverse=True)
    scores = [(tokenizer.decode(a), b) for a, b in scores]

    return probabilities, scores


In [21]:
template = tm3

In [75]:
#context = "David went to Beijing. Henry went to Sydney."
context = "David went to E. Henry went to F."
#answer_choices = ["Istanbul", "London", "Paris", "Perth"]
answer_choices = ["A", "B", "C", "D"]
question = "Where did John go?"

In [76]:
probabilities, scores = current_timestep_regular(context, answer_choices, question, template)

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [77]:
#context = "John went to Perth. Mary went to Paris."
#answer_choices = ["Istanbul", "London", "Paris", "Oslo"]
context = "John went to A. Mary went to B."
question = "Where did John go?"

In [78]:
entity_hidden_state = previous_timestep(context, "John")

new code working-modeling_t5


In [79]:
#context = "John David went to Beijing. Henry went to Sydney."
context = "John David went to E. Henry went to F."
#answer_choices = ["Istanbul", "London", "Paris", "Perth"]
answer_choices = ["A", "B", "C", "D"]
question = "Where did John go?"

In [80]:
probabilities_enhanced, scores_enhanced = current_timestep_enhanced(context, answer_choices, question, template, entity_hidden_state)

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [81]:
probabilities

{'A': 0.2807462158853593,
 'B': 0.3173507871874716,
 'C': 0.2608037882773989,
 'D': 0.14109920864977019}

In [82]:
probabilities_enhanced

{'A': 0.28986733557660793,
 'B': 0.3034951702718826,
 'C': 0.22740952436615747,
 'D': 0.17922796978535202}

In [43]:
for location in ["London", "Paris", "Oslo", "Istanbul", "Beijing", "Madrid", "Rome", "Sydney"]:
    loc_tokens = tokenizer.encode(location)
    print(location)
    print(loc_tokens)
    print()

London
[1524, 1]

Paris
[1919, 1]

Oslo
[29206, 1]

Istanbul
[20958, 1]

Beijing
[14465, 1]

Madrid
[12033, 1]

Rome
[7332, 1]

Sydney
[7476, 1]



In [6]:
for character in ["John", "Mary", "David", "Henry", "Lisa"]:
    char_tokens = tokenizer.encode(character)
    print(character)
    print(char_tokens)
    print()

John
[1079, 1]

Mary
[3790, 1]

David
[1955, 1]

Henry
[7780, 1]



# PART 1: Regular Prompt

In [4]:
tm = Template("""Read the following context and choose the best option to answer the
question.
Context: {{ context }}
Question: {{ question }}
Options:
 - {{ answer_choices | join("\n - ") }}""")

context = "John went to London. Mary went to Paris."
answer_choices = ["Istanbul", "London", "Paris", "Oslo"]
question = "Where was John?"
prompt = tm.render(context=context, question=question, answer_choices=answer_choices)
print(prompt)

Read the following context and choose the best option to answer the
question.
Context: John went to London. Mary went to Paris.
Question: Where was John?
Options:
 - Istanbul
 - London
 - Paris
 - Oslo


In [5]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")
out = model.generate(input_ids, max_new_tokens=10, return_dict_in_generate=True, output_scores=True)

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [9]:
tokenizer.decode(out.sequences[0])

'<pad> London</s>'

In [10]:
next_token_scores = torch.nn.functional.softmax(
                out.scores[0], dim=-1
            )  # (batch_size * num_beams, vocab_size)

loc_tokens = []
for location in answer_choices:
    loc_tokens.append(tokenizer.encode(location)[0])

probabilities = {}
for loc_token in loc_tokens:
    probabilities[tokenizer.decode(loc_token)] = next_token_scores[0][loc_token].item()

In [11]:
probabilities

{'Istanbul': 0.0004386534565128386,
 'London': 0.9951145648956299,
 'Paris': 0.0028165883850306273,
 'Oslo': 0.00011006197019014508}

# PART 2: Corrupted Run Probabilities

In [115]:
context = "David went to Beijing. Henry went to Sydney."
answer_choices = ["Istanbul", "London", "Paris", "Oslo"]
question = "Where did John go?"
prompt = tm3.render(context=context, question=question, answer_choices=answer_choices)
print(prompt)

David went to Beijing. Henry went to Sydney.
Where did John go?
Pick the correct answer from the following options:
- Istanbul
- London
- Paris
- Oslo


In [116]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")
out = model.generate(input_ids, max_new_tokens=10, return_dict_in_generate=True, output_scores=True)

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [117]:
tokenizer.decode(out.sequences[0])

'<pad> Istanbul</s>'

In [123]:
next_token_scores = torch.nn.functional.softmax(
                out.scores[0], dim=-1
            )  # (batch_size * num_beams, vocab_size)

loc_tokens = []
for location in answer_choices:
    loc_tokens.append(tokenizer.encode(location)[0])

probabilities = {}
for loc_token in loc_tokens:
    probabilities[tokenizer.decode(loc_token)] = next_token_scores[0][loc_token].item()
    
probabilities

{'Istanbul': 0.3860609829425812,
 'London': 0.3459107279777527,
 'Paris': 0.08937086910009384,
 'Oslo': 0.13198192417621613}

In [121]:
the_sum = sum(probabilities.values())
for key, value in probabilities.items():
    probabilities[key] = value/the_sum
    
probabilities

{'Istanbul': 0.4049628235119274,
 'London': 0.3628467814002619,
 'Paris': 0.09374653510601377,
 'Oslo': 0.13844385998179692}

In [122]:
sum(probabilities.values())

1.0

In [124]:
scores = []
for i, score in enumerate(next_token_scores[0]):
    scores.append( (i, score.item()) )

scores.sort(key=lambda x: x[1], reverse=True)
scores = [(tokenizer.decode(a), b) for a, b in scores]
scores

[('Istanbul', 0.3860609829425812),
 ('London', 0.3459107279777527),
 ('Oslo', 0.13198192417621613),
 ('Paris', 0.08937086910009384),
 ('Sydney', 0.014702122658491135),
 ('Beijing', 0.004248086363077164),
 ('Amsterdam', 0.0032933030743151903),
 ('Lisbon', 0.00274077826179564),
 ('Stockholm', 0.0019205965800210834),
 ('Constantin', 0.0011972648790106177),
 ('Moscow', 0.0010080903302878141),
 ('Singapore', 0.0009947175858542323),
 ('New', 0.0009905248880386353),
 ('Brussels', 0.0007955782348290086),
 ('Rome', 0.0007823242340236902),
 ('Copenhagen', 0.000773773470427841),
 ('Berlin', 0.0007724013994447887),
 ('Turkey', 0.0005210622330196202),
 ('Liverpool', 0.0005094882217235863),
 ('Bangkok', 0.0004906124668195844),
 ('Budapest', 0.00041919710929505527),
 ('Norway', 0.0003638784110080451),
 ('John', 0.00034062645863741636),
 ('Helsinki', 0.0003318880626466125),
 ('Madrid', 0.00032034219475463033),
 ('Tokyo', 0.0002911954652518034),
 ('Prague', 0.0002709285181481391),
 ('Oxford', 0.0002260

# PART 3: Move Character Embeddings

In [7]:
context = "John went to London. Mary went to Paris."
answer_choices = ["Istanbul", "London", "Paris", "Oslo"]
question = "Where did John go?"
prompt = tm3.render(context=context, question=question, answer_choices=answer_choices)
print(prompt)

John went to London. Mary went to Paris.
Where did John go?
Pick the correct answer from the following options:
- Istanbul
- London
- Paris
- Oslo


In [154]:
special_hidden[23]

tensor([[[ -60.5513,  -71.6467,  -47.6101,  ...,   17.4553,  -66.1806,
            45.4670],
         [ -22.8387, -136.3342,  -61.5524,  ...,  -26.1680,   30.3124,
            10.0684],
         [-100.0778,  -58.9518,   41.5424,  ...,   72.7503,   -3.3372,
             8.9747],
         ...,
         [  89.6812,   -7.5988,   75.5132,  ...,  -22.6045,   60.5369,
             9.2370],
         [  96.1115,   34.7098,   34.0191,  ...,  -99.0679,   66.2161,
           -15.0411],
         [  19.6166, -296.2766,   29.7596,  ...,  -51.6581,  103.2607,
            37.4480]]], grad_fn=<AddBackward0>)

In [155]:
out.last_hidden_state

tensor([[[-0.0991, -0.1015, -0.0699,  ...,  0.0299, -0.0869,  0.0632],
         [-0.0347, -0.1790, -0.0838,  ..., -0.0416,  0.0369,  0.0130],
         [-0.1488, -0.0758,  0.0554,  ...,  0.1133, -0.0040,  0.0113],
         ...,
         [ 0.1579, -0.0116,  0.1192,  ..., -0.0417,  0.0854,  0.0138],
         [ 0.1750,  0.0547,  0.0556,  ..., -0.1890,  0.0967, -0.0233],
         [ 0.0003, -0.0042,  0.0004,  ..., -0.0009,  0.0014,  0.0005]]],
       grad_fn=<MulBackward0>)

In [153]:
special_hidden[23] == out.last_hidden_state

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])

In [157]:
out.keys()

odict_keys(['last_hidden_state', 'special_hidden_states', 'hidden_states'])

In [160]:
out.hidden_states[24].shape

torch.Size([1, 37, 4096])

In [159]:
out.hidden_states[24] == out.last_hidden_state

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [166]:
37 * 4096

151552

In [168]:
#Adım2
for i in range(24):
    x = sum(sum(sum(out.hidden_states[i+1] == special_hidden[i])))
    if x != 151552:
        print(i)

23


In [172]:
#Adım4
for i in range(24):
    x = sum(sum(special_reformatted[i, :, :] == out.special_hidden_states[i][0, :, :]))
    if x != 151552:
        print(i)

In [177]:
#Adım5
for i in range(24):
    print(sum(entity_hidden_state[0, i, :] == out.special_hidden_states[i][0, 0, :]))

tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)


In [186]:
#Adım7
for i in range(24):
    print(sum(out.special_hidden_states[i][0, 0, :] == entity_hidden_state[0, i, :]))
print()
for i in range(24):
    print(sum(out.hidden_states[i+1][0, 0, :] == entity_hidden_state[0, i, :]))

tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)

tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(0)


In [180]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")
len_input_ids = len(input_ids[0])
out = model.encoder(input_ids, output_special=True, output_hidden_states=True, entity_hidden_states=entity_hidden_state, entity_inds=[0])
special_hidden = out.special_hidden_states # 24 x (1, T, d)
    
special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
for i, hidden in enumerate(special_hidden):
    special_reformatted[i:i+1, :, :] = hidden

new code working-modeling_t5


In [8]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")
len_input_ids = len(input_ids[0])
out = model.encoder(input_ids, output_special=True, output_hidden_states=True)
special_hidden = out.special_hidden_states # 24 x (1, T, d)
    
special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
for i, hidden in enumerate(special_hidden):
    special_reformatted[i:i+1, :, :] = hidden

new code working-modeling_t5


In [14]:
entity_ind = find_index_one(input_ids, "John", 0)
entity_hidden_state = special_reformatted[:, entity_ind, :].unsqueeze(0)

In [10]:
entity_ind

0

In [11]:
input_ids

tensor([[ 1079,   877,    12,  1524,     5,  3790,   877,    12,  1919,     5,
          2840,   410,  1079,   281,    58,  8356,     8,  2024,  1525,    45,
             8,   826,   931,    10,     3,    18, 20958,     3,    18,  1524,
             3,    18,  1919,     3,    18, 29206,     1]])

In [132]:
for i in range(24):
    print(sum(out.special_hidden_states[i][0, 0, :] == entity_hidden_state[0, i, :]))

tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)
tensor(4096)


In [12]:
context = "John David went to Beijing. Henry went to Sydney."
answer_choices = ["Istanbul", "London", "Paris", "Oslo"]
question = "Where did John go?"
prompt = tm3.render(context=context, question=question, answer_choices=answer_choices)
print(prompt)

John David went to Beijing. Henry went to Sydney.
Where did John go?
Pick the correct answer from the following options:
- Istanbul
- London
- Paris
- Oslo


In [15]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")
entity_inds = [ find_index_one(input_ids, "John", 0) ]
out = model.generate(input_ids=input_ids, entity_hidden_states=entity_hidden_state, entity_inds=entity_inds, max_new_tokens=1, return_dict_in_generate=True, output_scores=True)    

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [17]:
with open("to_compare.txt", "wb") as f:
    pickle.dump(out, f)

In [16]:
with open("to_compare.txt", "rb") as f:
    out2 = pickle.load(f)

In [17]:
out

GreedySearchEncoderDecoderOutput(sequences=tensor([[    0, 20958]]), scores=(tensor([[-59.4956, -14.5699, -17.2434,  ..., -58.4319, -59.1645, -58.8120]]),), encoder_attentions=None, encoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, decoder_hidden_states=None)

In [18]:
out2

GreedySearchEncoderDecoderOutput(sequences=tensor([[    0, 20958]]), scores=(tensor([[-57.6640, -14.1964, -16.7459,  ..., -56.5643, -57.3929, -56.9861]]),), encoder_attentions=None, encoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, decoder_hidden_states=None)

In [135]:
input_ids

tensor([[ 1079,  1955,   877,    12, 14465,     5,  7780,   877,    12,  7476,
             5,  2840,   410,  1079,   281,    58,  8356,     8,  2024,  1525,
            45,     8,   826,   931,    10,     3,    18, 20958,     3,    18,
          1524,     3,    18,  1919,     3,    18, 29206,     1]])

In [136]:
entity_inds

[0]

In [137]:
tokenizer.decode(out.sequences[0])

'<pad> Istanbul'

In [138]:
next_token_scores = torch.nn.functional.softmax(
                out.scores[0], dim=-1
            )  # (batch_size * num_beams, vocab_size)

loc_tokens = []
for location in answer_choices:
    loc_tokens.append(tokenizer.encode(location)[0])

probabilities = {}
for loc_token in loc_tokens:
    probabilities[tokenizer.decode(loc_token)] = next_token_scores[0][loc_token].item()

In [139]:
probabilities

{'Istanbul': 0.42404863238334656,
 'London': 0.3369309902191162,
 'Paris': 0.05942494049668312,
 'Oslo': 0.07412394136190414}

In [140]:
the_sum = sum(probabilities.values())
for key, value in probabilities.items():
    probabilities[key] = value/the_sum
    
probabilities

{'Istanbul': 0.4740470876764673,
 'London': 0.3766576342048662,
 'Paris': 0.06643157842408434,
 'Oslo': 0.08286369969458215}

In [141]:
sum(probabilities.values())

1.0

In [49]:
torch.argmax(next_token_scores[0])

tensor(14465)

In [50]:
next_token_scores

tensor([[1.4296e-24, 1.8635e-05, 8.5528e-07,  ..., 4.2936e-24, 1.8349e-24,
         2.8352e-24]])

In [54]:
scores = []
for i, score in enumerate(next_token_scores[0]):
    scores.append( (i, score.item()) )

scores.sort(key=lambda x: x[1], reverse=True)
scores = [(tokenizer.decode(a), b) for a, b in scores]
scores

In [60]:
scores

[('Beijing', 0.3729867935180664),
 ('Istanbul', 0.32160937786102295),
 ('London', 0.15457506477832794),
 ('Oslo', 0.06878328323364258),
 ('Paris', 0.030993076041340828),
 ('Sydney', 0.022291012108325958),
 ('Singapore', 0.001796290627680719),
 ('Bangkok', 0.0017849125433713198),
 ('Constantin', 0.0014752846909686923),
 ('John', 0.001429815893061459),
 ('Lisbon', 0.0014203054597601295),
 ('Amsterdam', 0.0013549844734370708),
 ('Brussels', 0.0012126025976613164),
 ('Bei', 0.001146177644841373),
 ('China', 0.0010128968860954046),
 ('Moscow', 0.0009832811774685979),
 ('Copenhagen', 0.0008368645794689655),
 ('Stockholm', 0.0007626041187904775),
 ('Berlin', 0.0006924326880834997),
 ('Seoul', 0.0005943586584180593),
 ('Budapest', 0.0005514992517419159),
 ('New', 0.0005007097497582436),
 ('Shanghai', 0.000472385436296463),
 ('Rome', 0.000407008541515097),
 ('Tokyo', 0.0004033115692436695),
 ('Helsinki', 0.00036624111817218363),
 ('Auckland', 0.00031367759220302105),
 ('Prague', 0.0003136156883

# Templates

In [6]:
tm1 = Template("""Read the following context and choose the best option to answer the question.
Context: {{ context }}
Question: {{ question }}
Options:
 - {{ answer_choices | join("\n - ") }}""")

tm2 = Template("""{{ context }}
{{ question }}
- {{ answer_choices | join("\n- ") }}""")

tm3 = Template("""{{ context }}
{{ question }}
Pick the correct answer from the following options:
- {{ answer_choices | join("\n- ") }}""")

tm4 = Template("""{{ context }}
According to the above context, choose the best option to answer the following question.
Question: {{ question }}
Options:
- {{answer_choices | join("\n - ")}}
""")

tm5 = Template("""{{ context }}
{{ question }}
Pick the best answer from the following options:
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

tm6 = Template("""{{ context }}
According to the above context, choose the best option to answer the following question.
Question: {{ question }}
Options:
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

tm7 = Template("""{{ context }}
{{ question }}
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

prompt_templates = [tm1, tm2, tm3, tm4, tm5, tm6, tm7]

# Toy Dataset

In [ ]:
previous_contexts = [
    "John went to London. Mary went to Paris. John loves Mary.",
    
]

current_contexts = [
    "David went to Beijing. Henry went to Sydney.",
    
]

questions = [
    "Where did John go?",
]

In [46]:
names = ["John", "Harry", "Andrew", "Lisa", "Mary", "Henry", "David", "Sophia", "Olivia", "Emma"]
for name in names:
    print(tokenizer.encode(name))
len(names)

[1079, 1]
[8929, 1]
[5954, 1]
[11712, 1]
[3790, 1]
[7780, 1]
[1955, 1]
[30174, 1]
[25051, 1]
[15325, 1]


10

In [83]:
cities = ["London", "Paris", "Oslo", "Istanbul", "Beijing", "Sydney", "Cairo", "Seoul", "Rome", "Prague"]

In [84]:
for name in cities:
    print(tokenizer.encode(name))
len(cities)

[1524, 1]
[1919, 1]
[29206, 1]
[20958, 1]
[14465, 1]
[7476, 1]
[28600, 1]
[28343, 1]
[7332, 1]
[23564, 1]


10

In [86]:
sentences = []
for name in names:
    for city in cities:
        sentences.append(name + " travelled to " + city + ".")
    

print(sentences)

['John travelled to London.', 'John travelled to Paris.', 'John travelled to Oslo.', 'John travelled to Istanbul.', 'John travelled to Beijing.', 'John travelled to Sydney.', 'John travelled to Cairo.', 'John travelled to Seoul.', 'John travelled to Rome.', 'John travelled to Prague.', 'Harry travelled to London.', 'Harry travelled to Paris.', 'Harry travelled to Oslo.', 'Harry travelled to Istanbul.', 'Harry travelled to Beijing.', 'Harry travelled to Sydney.', 'Harry travelled to Cairo.', 'Harry travelled to Seoul.', 'Harry travelled to Rome.', 'Harry travelled to Prague.', 'Andrew travelled to London.', 'Andrew travelled to Paris.', 'Andrew travelled to Oslo.', 'Andrew travelled to Istanbul.', 'Andrew travelled to Beijing.', 'Andrew travelled to Sydney.', 'Andrew travelled to Cairo.', 'Andrew travelled to Seoul.', 'Andrew travelled to Rome.', 'Andrew travelled to Prague.', 'Lisa travelled to London.', 'Lisa travelled to Paris.', 'Lisa travelled to Oslo.', 'Lisa travelled to Istanbul

In [ ]:
import sys
sys.path.append('/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/')
cp modeling_t5.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/t5/
cp utils.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/
cp modeling_outputs.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/